![](https://i.imgflip.com/1u8yz6.jpg)

# Hallo my beatiful Kagglers. 

Today is a non-competition day. Just lazy and calm diving through EDA river.  
I would like to apply some basic and advanced technics. 

As always, if you have any comments or improvements, please leave your thoughts below. 

**Libs and data import**

In [ ]:
#common
import numpy as np 
import pandas as pd 
import os
import re 
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

#visualisation
import seaborn as sns
sns.set_style('whitegrid')
#
import matplotlib.pyplot as plt
import matplotlib.style as style
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
#
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

#ml
from sklearn.metrics import SCORERS
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, recall_score, precision_score, r2_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report, precision_recall_curve
from sklearn.metrics import plot_confusion_matrix, mean_absolute_error, make_scorer
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, LabelEncoder, Binarizer, OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool, cv

In [ ]:
shrooms = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')

# Data analysys

In [ ]:
shrooms.info()

In [ ]:
shrooms.head().T

Thanks God, there are no missing values in dataset.  

We have 22 different features, which describes different sorts of mushrooms. 
We will try to create a classificator, which may give us an answer. Is this mushroom edible or not using only its characteristics. 

Start with edible classes.

In [ ]:
balance = shrooms['class'].value_counts().reset_index()

balance['index'].replace('e', 'edible', inplace=True)
balance['index'].replace('p', 'poisson', inplace=True)

fig = px.pie(balance,values="class",names="index",
             color_discrete_map={'poison':'firebrick',
                                 'edible':'cornflowerblue'})

fig.update_traces(textposition="inside", textinfo="value+percent+label")
fig.update_traces(textfont_size=18,
                  marker=dict(line=dict(color='#000000', width=2)))

fig.update(layout_showlegend=False)

fig.update_layout(
        title={'text': 'Distribution of edible and poison mushrooms',
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top',},
        font=dict(
            family="Helvetica",
            size=18))

fig.show()


Classes are closely equal. Don't need to do anything, just encode the values into 0 and 1. 

In [ ]:
shrooms['class'].replace('e',0, inplace=True)
shrooms['class'].replace('p', 1, inplace=True)

In [ ]:
shrooms.head()

To make the analysis and plots more intelligible, I will reconstruct all values to full names.

In [ ]:
shrooms_full = shrooms.copy()

shrooms_full = shrooms_full.replace({'cap-shape' : {'b':'bell','c':'conical','x':'convex','f':'flat','k':'knobbed','s':'sunken'}})
shrooms_full = shrooms_full.replace({'cap-surface' : {'f':'fibrous','g':'grooves','y':'scaly','s':'smooth'}})
shrooms_full = shrooms_full.replace({'cap-color' : {'n':'brown','b':'buff','c':'cinnamon','g':'gray','r':'green',
                                                    'p':'pink','u':'purple','e':'red','w':'white','y':'yellow'}})
shrooms_full = shrooms_full.replace({'bruises' : {'t':'bruises','f':'no'}})
shrooms_full = shrooms_full.replace({'odor' : {'a':'almond','l':'anise','c':'creosote','y':'fishy','f':'foul','m':'musty','n':'none',
                                              'p':'pungent','s':'spicy'}})
shrooms_full = shrooms_full.replace({'gill-attachment' : {'a':'attached','d':'descending','f':'free','n':'notched'}})
shrooms_full = shrooms_full.replace({'gill-spacing' : {'c':'close','w':'crowded','d':'distant'}})
shrooms_full = shrooms_full.replace({'gill-size' : {'b':'broad','n':'narrow'}})
shrooms_full = shrooms_full.replace({'gill-color' : {'k':'black','n':'brown','b':'buff','h':'chocolate','g':'gray','r':'green',
                                                    'o':'orange','p':'pink','u':'purple','e':'red','w':'white','y':'yellow'}})
shrooms_full = shrooms_full.replace({'stalk-shape' : {'e':'enlarging','t':'tapering'}})
shrooms_full = shrooms_full.replace({'stalk-root' : {'b':'bulbous','c':'club','u':'cup','e':'equal','z':'rhizomorphs',
                                                     'r':'rooted','?':'missing'}})
shrooms_full = shrooms_full.replace({'stalk-surface-above-ring' : {'f':'fibrous','y':'scaly','k':'silky','s':'smooth'}})
shrooms_full = shrooms_full.replace({'stalk-surface-below-ring' : {'f':'fibrous','y':'scaly','k':'silky','s':'smooth'}})
shrooms_full = shrooms_full.replace({'stalk-color-above-ring' : {'n':'brown','b':'buff','c':'cinnamon','g':'gray','o':'orange',
                                                                'p':'pink','e':'red','w':'white','y':'yellow'}})
shrooms_full = shrooms_full.replace({'stalk-color-below-ring' : {'n':'brown','b':'buff','c':'cinnamon','g':'gray','o':'orange',
                                                                'p':'pink','e':'red','w':'white','y':'yellow'}})
shrooms_full = shrooms_full.replace({'veil-type' : {'p':'partial','u':'universal'}})
shrooms_full = shrooms_full.replace({'veil-color' : {'n':'brown','o':'orange','w':'white','y':'yellow'}})
shrooms_full = shrooms_full.replace({'ring-number' : {'n':'none','o':'one','t':'two'}})
shrooms_full = shrooms_full.replace({'ring-type' : {'c':'cobwebby','e':'evanescent','f':'flaring','l':'large','n':'none',
                                                   'p':'pendant','s':'sheathing','z':'zone'}})
shrooms_full = shrooms_full.replace({'spore-print-color' : {'k':'black','n':'brown','b':'buff','h':'chocolate','r':'green','o':'orange',
                                                           'u':'purple','w':'white','y':'yellow'}})

shrooms_full = shrooms_full.replace({'population' : {'a':'abundant','c':'clustered','n':'numerous','s':'scattered','v':'several',
                                                     'y':'solitary'}})
shrooms_full = shrooms_full.replace({'habitat' : {'g':'grasses','l':'leaves','m':'meadows','p':'paths','u':'urban','w':'waste','d':'woods'}})


shrooms_full.head().T

Looks nice.  

For better understanding let's split the data into two subsets. Each for edible and poisonous mushrooms.

### NOTE 

Pandas method 'query()' do not support a column named 'class'.  

> shrooms.query('class==1')   
returns an error. keep this in mind!  

In this case you may use old but gold 'loc' method.

In [ ]:
edible = shrooms_full.loc[shrooms_full['class'] == 0]
poisonous = shrooms_full.loc[shrooms_full['class'] == 1]

edible.shape, poisonous.shape

In [ ]:
edible.head()

### Habitat 

My grandma has always said: 'Don't harvest mushrooms near roads'. Let's check the habitat of edible and poisonous mushrooms.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14,5), sharey=True)

sns.barplot(data = edible['habitat'].value_counts().reset_index(), 
            x='index', y='habitat', ax=axes[0], color='royalblue')

sns.barplot(data = poisonous['habitat'].value_counts().reset_index(), 
            x='index', y='habitat', ax=axes[1], color='firebrick')

axes[0].set_title('Edible', size=15)
axes[1].set_title('Poisonous', size=15)

axes[0].set_xlabel('')
axes[1].set_xlabel('')
axes[0].set_ylabel('Number of mushrooms')
axes[1].set_ylabel('')

plt.suptitle('Mushroom habitat', size=20, y=1.05);

Grandma was right. One thousand of mushrooms founded near paths are non-edible. So guys, please do not pick them up!

### Odor 

What do edible mushrooms smell?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14,5))

sns.barplot(y=edible['odor'].value_counts().index, x=edible['odor'].value_counts().values, color='royalblue', ax=axes[0])
sns.barplot(y=poisonous['odor'].value_counts().index, x=poisonous['odor'].value_counts().values, color='firebrick', ax=axes[1])

axes[0].set_title('Edible', size=15)
axes[1].set_title('Poisonous', size=15)

plt.suptitle('Difference in mushroom odor', size=20, y=1.05);

Never pick a mushroom smells foul or fishy. According to the dataset, edible shrooms smell nothing or have a nice almond or anise odour.

### Cap shape 

I suppose that edible mushrooms always have a round cap. And non-edible have a knobbed one. Let's check my hypothesis.

In [ ]:
edible_cap = edible.groupby('cap-shape', as_index=False)['odor'].count()
poisonous_cap = poisonous.groupby('cap-shape', as_index=False)['odor'].count()


fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['edible', 'poisonous'])

fig.add_trace(go.Pie(labels=edible_cap['cap-shape'], values=edible_cap['odor'],
                     name='edible'), 1, 1)

fig.add_trace(go.Pie(labels=poisonous_cap['cap-shape'], values=poisonous_cap['odor'],
                     name='poisonous'), 1, 2)

fig.update_traces(textinfo="value+percent+label", hoverinfo='label+percent', textfont_size=14,
                  marker=dict(line=dict(color='black', width=1)))

fig.update_layout(
        title={'text': 'Cap-shape difference',
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top',},
        font=dict(
            family="Helvetica",
            size=18))
fig.show()

Oops, I was wrong. Both edible and poisonous mushrooms commonly have the same cap shape. 
We will delete this column later to clear the dataset from non-informative features.

# A portrait of an average edible and poisonous mushroom   

Let's find the most common signs in each group? How many different attributes do we find between edible and poisonous mushrooms?

In [ ]:
common_features = pd.DataFrame(columns=['features','edible', 'poisonous', 'difference'])

for feature_name, row in zip(edible.columns[1:], range(edible.shape[1])):
    
    most_freq_edible = edible[feature_name].value_counts().index[0]
    most_freq_poisonous = poisonous[feature_name].value_counts().index[0]
    
    common_features.loc[row,'features'] = feature_name
    common_features.loc[row,'edible'] = most_freq_edible
    common_features.loc[row,'poisonous'] = most_freq_poisonous
    
    if most_freq_edible == most_freq_poisonous:
        common_features.loc[row,'difference'] = 'no'
    else:
        common_features.loc[row,'difference'] = 'yes'
        
        
    def highlight_background(s):
        return ['background-color: gold' if v == 'yes' else 'background-color: firebrick' for v in s]
    
    def color_text(val):
        color = 'black' if val == 'yes' else 'white'
        return 'color: %s' % color


common_features = common_features.sort_values(by='difference', ascending=False).reset_index(drop=True).\
                    style.\
                    applymap(color_text, subset='difference').\
                    apply(highlight_background, subset='difference')

common_features

If you are a 'mushroom novice', just like me, use this table. When picking up a shroom, check the first nine features first. 

## Machine learning  

Let's apply the CatBoost model to solve a classification problem.

In [ ]:
SEED = 42

In [ ]:
categorial = shrooms.columns[1:].tolist()
categorial

In [ ]:
cat_features = categorial

In [ ]:
X = shrooms.drop('class', axis=1)
y = shrooms['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=SEED)

In [ ]:
catboost = CatBoostClassifier(iterations=300,
                              learning_rate=0.1,
                              depth=4,
                              loss_function='Logloss',
                              subsample=0.8,
                              grow_policy='Depthwise',
                              verbose=100,
                              random_state=SEED
                              )

In [ ]:
catboost.fit(X_train, y_train, cat_features=categorial)

In [ ]:
importances = catboost.feature_importances_
column_names = X_train.columns.tolist()

In [ ]:
temp = pd.DataFrame(columns=['importances','columns'])

temp['importances'] = importances
temp['columns'] = column_names

In [ ]:
temp = temp.sort_values(by='importances', ascending=False)

In [ ]:
plt.figure(figsize=(6,15))

sns.barplot(x='importances', y='columns', data=temp, color='dimgray')

plt.title('Feature importances', size=18);

As you can see, odor and gill size are the most valuable features to prediction.

In [ ]:
pred = catboost.predict(X_test)

f1_score(y_test, pred)

In [ ]:
pred[:10]

In [ ]:
y_test[:10]

## Wow, we may predict is a mushroom edible just with this model!

Do enjoy.

**work_in_progress**